In [1]:
from satpy import Scene
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from metpy.plots import USCOUNTIES
from glob import glob
# pyproj is used to georeference GOES-16 latitude and longitude coordinates
from pyproj import Proj



ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the multiarray numpy extension module failed.  Most
likely you are trying to import a failed build of numpy.
Here is how to proceed:
- If you're working with a numpy git repository, try `git clean -xdf`
  (removes all files not under version control) and rebuild numpy.
- If you are simply trying to use the numpy version that you have installed:
  your installation is broken - please reinstall numpy.
- If you have already reinstalled and that did not fix the problem, then:
  1. Check that you are using the Python you expect (you're using C:\Users\Matthew\AppData\Local\conda\conda\envs\sats\python.exe),
     and that you have no directories in your PATH or PYTHONPATH that can
     interfere with the Python and numpy versions you're trying to use.
  2. If (1) looks fine, you can open a new issue at
     https://github.com/numpy/numpy/issues.  Please include details on:
     - how you installed Pyt

In [ ]:
from satpy import find_files_and_readers, Scene
from datetime import datetime
my_files = find_files_and_readers(base_dir='VIIRS',
                                  reader=('viirs_sdr', 'viirs_l1b'),
                                  start_time=datetime(2018, 11, 8, 0, 0, 0), end_time=datetime(2018, 11, 8, 23, 0, 0))
scn = Scene(filenames=my_files)
composite = 'true_color_lowres'
scn.load(['M04'])


In [ ]:
natscn = scn.resample(resampler='native')

natscn.show('true_color')

In [ ]:
plt.imshow(scn['C02'])


scn.load(['M04'])


my_area = scn['M04'].attrs['area'].compute_optimal_bb_area({'proj': 'lcc', 'lon_0': -121., 'lat_0': 25., 'lat_1': 25., 'lat_2': 25.})
new_scn = scn.resample(my_area)


crs = new_scn['M04'].attrs['area'].to_cartopy_crs()
ax = plt.axes(projection=crs)

ax.coastlines()
ax.gridlines()
ax.set_global()
plt.imshow(new_scn['I01'], transform=crs, extent=crs.bounds, origin='upper')

In [ ]:
myarea = scn['I02'].attrs['area'].compute_optimal_bb_area({'proj': 'lcc', 'lon_0': -121., 'lat_0': 25., 'lat_1': 25., 'lat_2': 25.})
new_scn = scn.resample(my_area)

In [ ]:
crs = scn['I02'].attrs['area'].to_cartopy_crs()
ax = plt.axes(projection=crs)

ax.coastlines()
ax.gridlines()
ax.set_global()
plt.imshow(scn['true_color'], transform=crs, origin='upper')

plt.show()



In [ ]:
hi_res = glob('ABI-L1b-RadC*.nc')
#red = ('C:/Users/Matthew/Documents/SJSUsummer2018/sat_stuff/single_channel_data/ABI-L1b-RadC_2018_312_14_OR_ABI-L1b-RadC-M3C02_G16_s20183121432181_e20183121434554_c20183121434591.nc')
lo_res = ('ABI-L2-MCMIPC_2018_312_14_OR_ABI-L2-MCMIPC-M3_G16_s20183121402181_e20183121404554_c20183121405068.nc')

red = xr.open_dataset('single_channel_data/ABI-L1b-RadC_2018_312_13_OR_ABI-L1b-RadC-M3C02_G16_s20183121357181_e20183121359554_c20183121359587.nc')

lo_res = xr.open_dataset(lo_res)

In [ ]:
red['Rad'].plot()

In [ ]:
# True Color RGB
#-----------------------------
# Load the three channels into appropriate R, G, and B variables
R = red['Rad'].data

# Apply range limits for each channel. RGB values must be between 0 and 1
R = np.clip(R, 0, 1)


# Apply a gamma correction to the image
gamma = 2.2
R = np.power(R, 1/gamma)




# Amount of contrast
contrast_amount = 105

# Apply contrast correction
R = contrast_correction(R, contrast_amount)
# Color tuple for pcolormesh
#-----------------------------
# Don't use the last column of the RGB array or else the image will be scrambled!
# This is the strange nature of pcolormesh.
r = R[:,:-1]

# Flatten the array, becuase that's what pcolormesh wants.
#colorTuple = r.reshape((r.shape[0] * r.shape[1]), 2)

# Adding an alpha channel will plot faster, according to Stack Overflow. Not sure why.
#colorTuple = np.insert(colorTuple, 3, 1.0, axis=1)



In [ ]:
def proj_GOES16():
    geostationaryProjParams_GOES16 = {'central_longitude' : -75.0,
                                    'satellite_height' : 35786023.0,
                                    'sweep_axis' : 'x'
                                   }
    return ccrs.Geostationary(**geostationaryProjParams_GOES16)

def contrast_correction(color, contrast):
    """
    Modify the contrast of an RGB
    See: #www.dfstudios.co.uk/articles/programming/image-programming-algorithms/image-processing-algorithms-part-5-contrast-adjustment/

    Input:
        C - contrast level
    """
    F = (259*(contrast + 255))/(255.*259-contrast)
    COLOR = F*(color-.5)+.5
    COLOR = np.minimum(COLOR, 1)
    COLOR = np.maximum(COLOR, 0)
    return COLOR

In [ ]:
# Satellite height
sat_h = red['goes_imager_projection'].perspective_point_height

# Satellite longitude
sat_lon = red['goes_imager_projection'].longitude_of_projection_origin

# Satellite sweep
sat_sweep = red['goes_imager_projection'].sweep_angle_axis

# The projection x and y coordinates equals the scanning angle (in radians) multiplied by the satellite height
# See details here: https://proj4.org/operations/projections/geos.html?highlight=geostationary
x = red['x'][:] * sat_h
y = red['y'][:] * sat_h

# Generate latitude/longitude grid
#-----------------------------
# Create a pyproj geostationary map object
p = Proj(proj='geos', h=sat_h, lon_0=sat_lon, sweep=sat_sweep)

# Perform cartographic transformation. That is, convert image projection coordinates (x and y)
# to latitude and longitude values.
XX, YY = np.meshgrid(x, y)
lons, lats = p(XX, YY, inverse=True)

In [ ]:
geostationaryProjParams_GOES16 = {'central_longitude' : -75.0,
                                    'satellite_height' : 35786023.0,
                                 'sweep_axis' : 'x'
                                   }
#geostationaryProjParams_GOES16
proj = ccrs.Geostationary(**geostationaryProjParams_GOES16)
#proj = proj_GOES16()

In [ ]:
plt.figure(figsize=(10, 8))
ax = plt.axes(projection=proj)

ax.imshow(R, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()))

ax.coastlines(resolution='50m', color='w')

In [ ]:
# Create a Cartopy projection object

lccProjParams_HRRR = { 'central_latitude'   : -38.5,        # same as lat_0 in proj4 string 
                       'central_longitude'  : -97.5,        # same as lon_0
                       'standard_parallels' : (38.5, 38.5), # same as (lat_1, lat_2)
                     }

HRRR_proj = ccrs.LambertConformal(**lccProjParams_HRRR)

In [ ]:
# GOES on HRRR Projection
fig = plt.figure(figsize=(8,8))

# "destination" projection
ax = plt.axes(projection=HRRR_proj)

# "Source" projection
ax.pcolormesh(lons, lats, R, cmap='Greys_r', transform=ccrs.PlateCarree())

ax.coastlines(resolution='10m', color='dodgerblue')

#ax.set_global()

ax.gridlines()

In [ ]:
# GOES on HRRR projection
fig = plt.figure(figsize=(16,9))

# "destination" projection
ax = plt.axes(projection=HRRR_proj)

# "Source" projection
newmap = ax.pcolormesh(lons, lats, R, color=colorTuple, linewidth=0, transform=ccrs.PlateCarree())
newmap.set_array(None)

ax.coastlines(resolution='10m', color='dodgerblue')

ax.set_extent([-122, -120, 38.5, 40.5])
ax.gridlines()



In [ ]:
# Load the three channels into appropriate R, G, and B variables
R_f = C['CMI_C07'].data
G_f = C['CMI_C06'].data
B_f = C['CMI_C05'].data
# Normalize each channel by the appropriate range of values  e.g. R = (R-minimum)/(maximum-minimum)
R_f = (R_f-273)/(333-273)
G_f = (G_f-0)/(1-0)
B_f = (B_f-0)/(0.75-0)

# Apply range limits for each channel. RGB values must be between 0 and 1
R_f = np.clip(R_f, 0, 1)
G_f = np.clip(G_f, 0, 1)
B_f = np.clip(B_f, 0, 1)

# Apply the gamma correction to Red channel.
#   corrected_value = value^(1/gamma)
gamma = 0.4
R_f = np.power(R_f, 1/gamma)

# The final RGB array :)
RGB_f = np.dstack([R_f, G_f, B_f])


In [ ]:


# Create a color tuple for pcolormesh

# Don't use the last column of the RGB array or else the image will be scrambled!
# This is the strange nature of pcolormesh.
rgb_f = RGB_f[:,:-1,:]

# Flatten the array, becuase that's what pcolormesh wants.
colorTuple_f = rgb_f.reshape((rgb_f.shape[0] * rgb_f.shape[1]), 3)

# Adding an alpha channel will plot faster, according to Stack Overflow. Not sure why.
colorTuple_f = np.insert(colorTuple_f, 3, 1.0, axis=1)



In [ ]:
# GOES on HRRR projection
fig = plt.figure(figsize=(16,9))

# "destination" projection
ax = plt.axes(projection=HRRR_proj)

# "Source" projection
newmap = ax.pcolormesh(lons, lats, R_f, color=colorTuple_f, linewidth=0, transform=ccrs.PlateCarree())
newmap.set_array(None)

ax.coastlines(resolution='10m', color='dodgerblue')

ax.set_extent([-122, -120, 38.5, 40.5])
ax.gridlines()

In [ ]:
#Load the three channels into appropriate R, G, and B variables
TC_R = C['CMI_C02'].data
TC_G = C['CMI_C03'].data
TC_B = C['CMI_C01'].data

# Apply range limits for each channel. RGB values must be between 0 and 1
TC_R = np.clip(TC_R, 0, 1)
TC_G = np.clip(TC_G, 0, 1)
TC_B = np.clip(TC_B, 0, 1)

# Apply a gamma correction to the image
gamma = 2.2
TC_R = np.power(TC_R, 1/gamma)
TC_G = np.power(TC_G, 1/gamma)
TC_B = np.power(TC_B, 1/gamma)

# Calculate the "True" Green
TC_G_true = 0.45 * TC_R + 0.1 * TC_G + 0.45 * TC_B
TC_G_true = np.maximum(TC_G_true, 0)
TC_G_true = np.minimum(TC_G_true, 1)

# The RGB array for the true color image
TC_RGB = np.dstack([TC_R, TC_G_true, TC_B])

# Create a color tuple for pcolormesh
# Amount of contrast
contrast_amount = 150

# Apply contrast correction
TC_RGB_contrast = contrast_correction(np.dstack([TC_R, TC_G_true, TC_B]), contrast_amount)
# Don't use the last column of the RGB array or else the image will be scrambled!
# This is the strange nature of pcolormesh.
TC_rgb = TC_RGB[:,:-1,:]


# Flatten the array, becuase that's what pcolormesh wants.
TC_colorTuple = TC_rgb.reshape((TC_rgb.shape[0] * TC_rgb.shape[1]), 3)

# Adding an alpha channel will plot faster, according to Stack Overflow. Not sure why.
TC_colorTuple = np.insert(TC_colorTuple, 3, 1.0, axis=1)

In [ ]:
from metpy.plots import USCOUNTIES # Make sure metpy is updated to latest version.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6), subplot_kw={'projection': ccrs.PlateCarree()})



newmap = ax1.pcolormesh(lons, lats, R_f, color=colorTuple_f, linewidth=0, transform=ccrs.PlateCarree())
newmap.set_array(None)
ax1.coastlines(resolution='10m', color='dodgerblue')
ax1.add_feature(USCOUNTIES.with_scale('500k'), edgecolor='dodgerblue', linewidth=.2)
ax1.set_extent([-122.5, -120, 38, 41])


ax1.set_title('Fire Temperature');



ax2.pcolormesh(lons, lats, R, color='Greys_r', linewidth=0, transform=ccrs.PlateCarree())

ax2.coastlines(resolution='10m', color='dodgerblue')
ax2.add_feature(USCOUNTIES.with_scale('500k'), edgecolor='dodgerblue', linewidth=.2)
ax2.set_extent([-122.5, -120, 38, 41])



ax2.set_title('True Color');
plt.savefig('fireT_truecolor.png', dpi = 500)

In [ ]:
# Composite the two RGB images by finding the maximum R, G, and B value between the two
# Then create the new RGB and colorTuple
composite_R = np.maximum(R, TC_R)
composite_G = np.maximum(G, TC_G_true)
composite_B = np.maximum(B, TC_B)

composite_RGB = np.dstack([composite_R, composite_G, composite_B])

# Create a color tuple for pcolormesh

# Don't use the last column of the RGB array or else the image will be scrambled!
# This is the strange nature of pcolormesh.
composite_rgb = composite_RGB[:,:-1,:]

# Flatten the array, becuase that's what pcolormesh wants.
composite_colorTuple = composite_rgb.reshape((composite_rgb.shape[0] * composite_rgb.shape[1]), 3)

# Adding an alpha channel will plot faster, according to Stack Overflow. Not sure why.
composite_colorTuple = np.insert(composite_colorTuple, 3, 1.0, axis=1)

In [ ]:
# GOES on HRRR projection
fig = plt.figure(figsize=(16,9))

# "destination" projection
ax = plt.axes(projection=HRRR_proj)

# "Source" projection
newmap = ax.pcolormesh(lons, lats, R, color=composite_colorTuple, linewidth=0, transform=ccrs.PlateCarree())
newmap.set_array(None)

ax.coastlines(resolution='10m', color='dodgerblue')

ax.set_extent([-122.5, -120, 38, 41])
ax.add_feature(USCOUNTIES.with_scale('500k'), edgecolor='dodgerblue', linewidth=.2)

plt.savefig('fire+truecolot.png', dpi = 500)